<a href="https://colab.research.google.com/github/Lynneice/hello-world/blob/master/McNutt_Fine_Tuning_BERT_for_Email_Address_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pipeline intended to enrich email data post-APRIMO sunset.



*   audience_member_id flags extremely informative, underage/ooverage and other email eligiblity conditions.
*   PEGA may not inherit exact same taxonomy, but if we can build out the next iteration of audience_member_id flags, PEGA may see better performance.
*  With text-based classification in place, the goal is to enrich lead profile with additional signals based on information mined from email address alone.



In [ ]:
# Install necessary packages
# !pip install transformers datasets scikit-learn pandas sklearn2pmml


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn2pmml import sklearn2pmml  # Make sure this is imported
from sklearn2pmml.pipeline import PMMLPipeline

In [ ]:

# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch


In [ ]:
# dummy dataset
# data = {
#     'email': [
#         'john.doe@gmail.com',
#         'student@harvard.edu',
#         'bot123@unknown.xyz',
#         'tester@agency.gov',
#         'vip@partner.com'
#     ],
#     'label': [0, 1, 2, 3, 4]  # Corresponding labels
# }
PATH = "/content/email_classifier_model/training data/fake_emails_8k.csv"
df = pd.read_csv(PATH).sample(800, random_state=42)


# df = pd.DataFrame(data)


# # Split into training and test sets
# train_texts, test_texts, train_labels, test_labels = train_test_split(
#     df['email'], df['label'], test_size=0.2, random_state=42
# )


# Create a new column with domain information
df['domain'] = df['email'].str.split('@').str[1]

# Split into training and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df[['email', 'domain']], df['label'].values, test_size=0.2, random_state=42
)

In [ ]:
# # Initialize tokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# # Tokenize the data
# train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
# test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)

# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize email and domain separately
def tokenize_function(examples):
    email_encodings = tokenizer(examples['email'], truncation=True, padding=True)
    domain_encodings = tokenizer(examples['domain'], truncation=True, padding=True)

    # Combine encodings (you might need to adjust how you combine them)
    return {**email_encodings, **{k + '_domain': v for k, v in domain_encodings.items()}}

# Apply tokenization
train_encodings = tokenize_function(train_texts.to_dict(orient='list'))
test_encodings = tokenize_function(test_texts.to_dict(orient='list'))

In [ ]:
# Split into training and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['email'], df['label'].values, test_size=0.2, random_state=42
)

In [ ]:
# Create a custom dataset class
# class EmailDataset(torch.utils.data.Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels

#     def __getitem__(self, idx):
#         item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#         item['labels'] = torch.tensor(self.labels[idx])
#         return item

#     def __len__(self):
#         return len(self.labels)

class EmailDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create dataset objects
train_dataset = EmailDataset(train_encodings, train_labels)
test_dataset = EmailDataset(test_encodings, test_labels)


Model Initialization

In [ ]:
# Load pre-trained BERT model with a classification head
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training Setup

In [ ]:
# define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    # Replace 'evaluation_strategy' with 'eval_accumulation_steps' if using an older version
    # evaluation_strategy="epoch",
    eval_accumulation_steps=1,  # Evaluate at the end of each epoch
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
)

# Initialize Tranier
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer
)

<ipython-input-19-dc7ae5072f8e>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


traininng setup

In [ ]:
#train the model
trainer.train()

#evaluate
trainer.evaluate()

Step,Training Loss
10,1.484400
20,0.998900
30,0.969500
40,0.576300
50,0.349700
60,0.151300
70,0.050100
80,0.026900
90,0.012400
100,0.008000


{'eval_loss': 0.0008769541746005416,
 'eval_runtime': 13.0634,
 'eval_samples_per_second': 12.248,
 'eval_steps_per_second': 3.062,
 'epoch': 3.0}

In [ ]:
#save the model and tokenizer
model.save_pretrained('./email_classifier_model')
tokenizer.save_pretrained('./email_classifier_model')


('./email_classifier_model/tokenizer_config.json',
 './email_classifier_model/special_tokens_map.json',
 './email_classifier_model/vocab.txt',
 './email_classifier_model/added_tokens.json')

Exporting to PMML - PEGA accepts pmml format but bert is huge lol </br>
in lieu of wrappnig BERT models with SKL consider using formats like ONNX or TorchScript

In [ ]:
# !pip install scikit-learn==1.5.2

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn2pmml import sklearn2pmml
from sklearn2pmml.pipeline import PMMLPipeline

# Create a simple pipeline
vectorizer = CountVectorizer()
classifier = LogisticRegression()

pipeline = PMMLPipeline([
    ("vectorizer", vectorizer),
    ("classifier", classifier)
])

# Fit the pipeline on the data
pipeline.fit(df['email'], df['label'])

# Export to PMML
sklearn2pmml(pipeline, "email_classifier.pmml", with_repr=True)


In [ ]:
# using the saved model
from transformers import BertTokenizer, BertForSequenceClassification

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('./email_classifier_model')

# Load the model
model = BertForSequenceClassification.from_pretrained('./email_classifier_model')

# Set the model to evaluation mode
model.eval()





BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
# example email address
email = "johsnow@google.gurg"

# Tokenize the input
inputs = tokenizer(email, return_tensors="pt", truncation=True, padding=True)



with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# get the predicted class
predicted_class = logits.argmax().item()
print(f"Predicted class: {predicted_class}")

Predicted class: 0


map classes

In [ ]:
# Define your label mapping
label_mapping = {
    0: "VALID",
    1: "EDU",
    2: "BOT / BAD ACTOR / SUSPICIOUS",
    3: "TEST / INTERNAL (.mil, .gov, etc.)",
    4: "ON LIST / Known Member"
}

# Get the label
predicted_label = label_mapping[predicted_class]
print(f"Predicted label: {predicted_label}")


Predicted label: VALID


In [ ]:
from transformers import pipeline

# Initialize the pipeline
classifier = pipeline("text-classification", model='./email_classifier_model', tokenizer='./email_classifier_model')

# Classify the email
result = classifier("butt@los.gov")
print(result)


Device set to use cpu


[{'label': 'LABEL_3', 'score': 0.9957094192504883}]


In [ ]:
from transformers import pipeline
import torch.nn.functional as F

# Initialize the pipeline
classifier = pipeline("text-classification", model='./email_classifier_model', tokenizer='./email_classifier_model', return_all_scores=True)

# Classify the email and get scores for all classes
result = classifier("example@university.edu")
print(result)

Device set to use cpu


[[{'label': 'LABEL_0', 'score': 0.00020143704023212194}, {'label': 'LABEL_1', 'score': 0.9990304708480835}, {'label': 'LABEL_2', 'score': 0.00020884840341750532}, {'label': 'LABEL_3', 'score': 0.0002545579627621919}, {'label': 'LABEL_4', 'score': 0.00030479850829578936}]]


/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
result = classifier("marymary@ilstu.edu")
print(result)

[{'label': 'LABEL_1', 'score': 0.9990909099578857}]


In [ ]:
result = classifier("john.snow@rapp.com")
print(result)

[{'label': 'LABEL_0', 'score': 0.9992889165878296}]


In [ ]:
result = classifier("mary.smith@grif.edu")
print(result)

[{'label': 'LABEL_1', 'score': 0.9990668892860413}]
